In [1]:
import os

In [60]:
from functools import reduce

In [3]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [2]:
from config import spark

22/05/03 10:23:49 WARN Utils: Your hostname, eniac resolves to a loopback address: 127.0.1.1; using 192.168.100.6 instead (on interface wlp3s0)
22/05/03 10:23:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/03 10:23:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/03 10:23:50 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/05/03 10:23:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
ref = spark.read.csv('ibp_data', header=True)

In [7]:
preprocess = '/media/fabio/19940C2755DB566F/PAMepi/datalake/DATALAKE/2022-04-19/preprocess/'

In [8]:
vac = spark.read.csv(os.path.join(preprocess, 'vacinacao'), header=True)

In [9]:
srag = spark.read.csv(os.path.join(preprocess, 'srag'), header=True)

In [11]:
sg = spark.read.csv(os.path.join(preprocess, 'sindrome_gripal'), header=True)

In [12]:
wcota = spark.read.csv(os.path.join(preprocess, 'wcota'), header=True)

In [14]:
mobility = spark.read.csv(os.path.join(preprocess, 'google_mobility'), header=True)

In [17]:
ref = ref.withColumnRenamed('code', 'code_mun_ibge')

In [37]:
wcota = wcota.withColumn('code_mun_ibge', F.regexp_extract('code_mun_ibge', '\d{6}', 0))

In [48]:
ref.columns

['dates', 'Cod', 'Nome', 'Pop', 'code_mun_ibge', 'UF', 'ip_vl_n']

In [65]:
ref = ref.withColumnRenamed('Cod', 'code_full_mun_ibge') \
    .withColumnRenamed('Nome', 'mun_name') \
    .withColumnRenamed('UF', 'uf') \
    .withColumnRenamed('Pop', 'pop') \
    .withColumnRenamed('dates', 'date')

In [66]:
ref = ref.withColumn('uf_code', F.regexp_extract('code_mun_ibge', '\d{2}', 0))

In [67]:
ref.show(1, truncate=False)

+----------+------------------+--------------------------+-----+-------------+---+-------+-------+
|date      |code_full_mun_ibge|mun_name                  |pop  |code_mun_ibge|uf |ip_vl_n|uf_code|
+----------+------------------+--------------------------+-----+-------------+---+-------+-------+
|2022-01-12|1100015           |Alta Floresta D'Oeste (RO)|22516|110001       |RO |0.086  |11     |
+----------+------------------+--------------------------+-----+-------------+---+-------+-------+
only showing top 1 row



In [71]:
mobility.show(1, vertical=True)

-RECORD 0----------------------------------------------------------------
 date                                               | 2021-03-31         
 mun_name                                           | São José da Tapera 
 uf                                                 | Alagoas            
 retail_and_recreation_percent_change_from_baseline | 0                  
 grocery_and_pharmacy_percent_change_from_baseline  | 0                  
 parks_percent_change_from_baseline                 | 0                  
 transit_stations_percent_change_from_baseline      | 0                  
 workplaces_percent_change_from_baseline            | 1                  
 residential_percent_change_from_baseline           | 0                  
only showing top 1 row



In [68]:
result = reduce(lambda df1, df2: df1.join(df2, how='left', on=['date', 'code_mun_ibge']), [ref, vac, sg, srag, wcota])

In [74]:
result.join(mobility, on=['date', 'mun_name', 'uf'], how='left').orderBy('date').coalesce(1).write.csv(os.path.join('/media/fabio/19940C2755DB566F/PAMepi/datalake/DATALAKE/2022-04-19/', 'result'), header=True)